# CCS 2.0 

*(You'll need `pyrsistent`, `networkx` and `pygraphviz`. You'll also need graphviz.)*

### Parsing rules

We'll be parsing and dealing with the actual CCS syntax, so we need to have access to the parser, and to understand a bit about how the AST represents rules.

In [ ]:
import sys
sys.path.append("src")
from ccs import parser

Most of the syntax is "primitive" rules: property settings, additional context settings, imports, etc. Those are uninteresting, uniformly represented as leaf nodes of the AST. Normal rules with selectors (whether actually nested or not) are represented as nested rule-sets, each an optional selector and a list of nested rules, either primitive or further nested.

Each selector is represented as an arbitrary boolean expression containing literals, conjunctions and disjunctions.

In [ ]:
import io
def parse_only(string):
    return parser.Parser().parse(io.StringIO(string), '-')

In [ ]:
print(parse_only("""
a, f b e, c { c d { x = y } e f { foobar = abc }}
"""))

In [ ]:
print(parse_only("""
a f, b e, c { c d {}}
"""))

In [ ]:
print(parse_only("""
(a, b, c) (d, e, f) { g, h, i {}}
"""))

It'll be handy in the below to be able to parse just the expression language:

In [ ]:
def parse_expr(string):
    return parser.Parser().parse_selector(io.StringIO(string))

In [ ]:
print(parse_expr("(a, b, c) (d, e, f)"))

### Matching

The problem of matching rules to contexts amounts to testing formulae against truth-value assignments. But there are a few particularities which apply to our setting...

 - we want to evaluate rules under partial assignments as well as total assignemnts, identifying
   rules which are not yet satisfied, but remain satisfiable by future assignments.
 - we're always monotonically extending partial assignments and then re-evaluating, so it seems 
   like a good idea to re-use the current state as a starting point for future evaluations.
 - many assignments will co-exist at the same time, and each may subsequently be used as the
   starting point for future evaluations.
   
If we use an algorithm that just freshly evaluates all the rules against each assignment from scratch, there's not much more to do. If, however, we want to use an algorithm that performs work incrementally from the prior assignment, then we also want persistent data structures so that the old and new states can co-exist without copying.

Here are some ideas...

#### Brute force

 1. Index rules by properties set in that context
 2. Do nothing else until a property is requested
 3. Find all rules setting that property
 4. Evaluate each rule against the current assignment
 5. Apply specificity logic, etc.
 
This might be just fine, particularly if there are lots of different properties with only a few settings each. For situations where lots of properties are queried and set in the same context, there are some simple tricks that could speed things up. For instance, a given rule could be marked as satisfied (or not) by a given assignment so that future properties queried in the same context wouldn't need to reevaluate those rules. But in the case where are many, many settings of the same property in a large number of different rules, this seems unavoidably pretty bad.

There maybe other use cases for which this approach wouldn't work as well. For instance, enumerating properties set in a given context, or enumerating all the possible values of a given key that could be added to reveal additional settings. Those are more rare, though, and it would be ok if they were more expensive, as long as they're tractable.

#### DAG/Rete

 1. Use rules to build an immutable graph, with root nodes for each literal
 2. As the context is extended with additional facts, propagate those facts through the graph,
    activating child nodes as appropriate
 3. When a node containing property settings is activated, add those settings to the properties
    visible in the current context
 4. Apply specificity logic, either separately or in step 3 when adding settings
 
This is most similar to the implementation of CCS1. Given the requirements above, the node activation state must be kept separately from the main graph, and in a persistent data structure. Likewise the visible settings.

Querying properties and enumerating properties are both trivial. The problem of enumerating key values remains tricky, but I've already implemented one variant and it's not too bad.

There are lots of variants of this involving different approaches to building the dag. Since we're dealing with DNF terms, one possibility is to build a graph for disjunctions and one for conjunctions (or, equivalently, a single graph in two layers). The only thing remaining then is to optimize the structure of each of those graphs to amortize work done at each step.

#### Other matching approaches

There are other matching approaches to indexing and matching boolean formulae. Perhaps one of these would be better, but it would need to support partial matching (to find satisfiable but not yet satisfied rules), and may need to support an incremental implementation using immutable data structures, depending on its overall performance.

#### CCS 2

For now, we use a DAG approach different from that in CCS1. Let's see how it works...

In CCS1, we directly built dag nodes from this AST as we traversed it in a single pass. That won't work for CCS2, so we'll need to convert the AST to a collection of complete expressions, each with its list of primitive rules. In the process of doing this, we also normalize the expressions, a topic to which we now turn.

### Representing and minimizing rules

Ultimately, our rules are represented as positive monotone formulae in DNF. This allows us to define specificity in a natural way, affords us a normal form suitable for diffing, and gives a uniform starting point for matching. Typical rules are actually closer to CNF, unfortunately, but with a couple of tricks, we can avoid an exponential blowup in the common cases.

Because of the extra invariants, and because we'll be using these objects for matching, we represent them separately from the `Expr` type in the AST.

A _clause_ is a set of literals, and a _formula_ a set of clauses. Formulae also have a slot for a set of _shared sub-clauses_, clauses known to be subclauses of at least two top-level clauses in the formula itself. The purpose of this will be clear later.

Clauses and formulae are sorted first by size, then lexicographically. The purpose of sorting by size will be clear later as well. Other than that, we just need a bunch of boilerplate functionality. See the source file for details.

In [ ]:
from ccs.formula import Clause, Formula

In [ ]:
Formula([Clause({'a', 'b', 'c'}), Clause({'ab', 'e'}), Clause({'a', 'ab', 'c'})])

For any formula, we define a normal form which exists, is unique, and is equivalent to the original formula under the usual interpretation of boolean logic.

Clauses are always normal, since all literals are positive. Formulae are normalized by removing any clause subsumed by any other. A clause $c$ is _subsumed_ by a clause $s$ if $s \subseteq c$. This is the obvious $O(mn)$ algorithm. Our formulae are usually of size 1, so this is just fine.

**TODO**: In fact, clauses could be further normalized, using the additional semantics of literals:

    x.a x ... -> x.a ...
    x.a x.!b ... -> x.a ...
    x.!b x ... -> x.!b ...
    x.a x.!a ... -> false (prune clause)
    x.a x.b ... -> false (prune clasue)

These are all relatively unlikely to be written by hand, though, so it may not be worth bothering.

In [ ]:
from ccs.formula import normalize

In [ ]:
form = Formula([Clause(['a', 'b']),
                Clause(['b']),
                Clause(['a']),
                Clause(['c', 'd']),
                Clause(['a', 'c', 'd']),
                Clause(['c', 'd'])])
form

In [ ]:
normalize(form)

### Getting to DNF

It's much more convenient to specify rules as arbitrary boolean expressions (as is allowed in CCS1), particularly since in practice most things we want to express are closer to CNF than to DNF. We'll convert them to DNF, and can report an error if a formula expands beyond a fixed maximum size. We'll use a trick to handle the most common case of nested disjunctions, when we have a number of possible values for the same key. In a rule like this:

    (key.val1, key.val2, key.val3) x y z

which is by far the most common use of disjunction, there's really no need for us to expand it into three separate clauses. The specificity will be the same in any case, and we'll be matching literals by looking up the keys in a hash table anyway, in order to handle negative matches. So this doesn't add any complexity there either. The only thing we sacrifice is full normalization for purposes of diffing. Note that this is driven purely by hints in the input: we don't attempt to discover any new simplifications of this form.

**TODO**: really verify this code and make sure this is really a good idea...

We also notice and remember sharing introduced during DNF-conversion, so that it can be exploited later while building the DAG: whenever we're doing an `expand()` step, we gather up all the one-element children first and add a shared subclause containing those elements, and then expand all the multi-element children.

`to_dnf()` converts an AST `Expr` to a normalized `Formula`.

In [ ]:
from ccs.ast import Expr, Op, Step
from ccs.dag import Key

In [ ]:
from ccs.ast import flatten
from ccs.dnf import to_dnf

In [ ]:
s = to_dnf(parse_expr("a b, c d"))
print(s.shared)
s

In [ ]:
s = to_dnf(parse_expr("(a, b) (c, d)"))
print(s.shared)
s

In [ ]:
s = to_dnf(flatten(parse_expr("(a b) (c d)")))
print(s.shared)
s

In [ ]:
s = to_dnf(flatten(parse_expr("(a f (b, e)) (c d)")))
print(s.shared)
s

In [ ]:
s = to_dnf(flatten(parse_expr("a, (a f (b, e)) (c d)")))
print(s.shared)
s

In [ ]:
s = to_dnf(parse_expr("(a b) (c, d)"))
print(s.shared)
s

In [ ]:
s = to_dnf(flatten(parse_expr("(a, b), c d")))
print(s.shared)
s

In [ ]:
s = to_dnf(parse_expr("(a, b, c) (d, e, f) (g, h, i)"))
print(s.shared)
s

In [ ]:
s = to_dnf(flatten(parse_expr("a b (c q, d) (e r, f)")))
print(s.shared)
s

We can allow the user to override a default limit to allow exploding expansions up to whatever size they really need.

In [ ]:
try:
    to_dnf(parse_expr("(a, b, c) (d, e, f) (g, h, i)"), limit=20)
except ValueError as e:
    print(e)

Here's an example where normalization is important, we'll revisit this when we look at specificity:

In [ ]:
to_dnf(parse_expr("(a, b) (a, c)"))

### Processing rulesets

With that out of the way, let's return to processing a whole ruleset. As mentioned above, we'll traverse the entire AST, combining each partial expression with its normalized parent to produce a new tree with each node having a complete normalized formula, its children, and any primitive rules defined at that node.

In [ ]:
from ccs.rule_tree import RuleTreeNode

It'll be nice to be able to visualize these, so let's do that now.

In [ ]:
import networkx as nx
from IPython.display import Image
    
def draw_tree(tree):
    G = nx.DiGraph()
    G.graph['dpi'] = 60
    def add_node(n):
        l = n.label()
        G.add_node(n, label=l if len(l) else "-", shape='box', style="filled", fillcolor=n.color())
        for c in n.children:
            G.add_edge(n, c)
            add_node(c)
    add_node(tree)
    return Image(nx.nx_agraph.to_agraph(G).draw(prog='dot',format='png'))

Let's see an example:

In [ ]:
n = parse_only("""
a, f b e, c {
  c d {
    x = y
  }
  e f {
    foobar = abc
  }
}
a, c, b e f : baz = quux
""")
root = RuleTreeNode()
n.add_to(root)
draw_tree(root)

Note that there's no deduplication, pruning of empty nodes, etc. The structure just mirrors exactly the structure of the input.

###  The DAG

Time to see how the rule dag is structured! The dag will be rooted with a bunch of literals, which in actual practice consist of key/value-pattern matchable forms, but for now we'll just leave them opaque.

Nodes consist of an operation (and/or), an activation tally count, their children and their property settings. The top-level dag contains an otherwise unused node for the root-level settings.

In [ ]:
from ccs.dag import AndNode, LiteralMatcher, OrNode

We'll want to be able to visualize these things too, so let's get that out of the way now. The second argument `t` will be used when we use the dags for matching below...

In [ ]:
# TODO doesn't this visit the same nodes over and over??
# t will be used later...
def draw_dag(dag, t={}, active_only=False):
    G = nx.DiGraph()
    G.graph['dpi'] = 60
    G.graph['rankdir'] = 'BT'
    def add_nodes(p, ns):
        for n in ns:
            if active_only and n not in t:
                continue
                
            if isinstance(n, OrNode):
                label = 'V'
                if n in t:
                    color = 'palegreen'
                else:
                    color = 'lightblue'
            else:
                count = t.get(n, n.tally_count)
                label = str(n.tally_count)
                if count == 0:
                    color = 'palegreen'
                elif count != n.tally_count:
                    color = 'mistyrose'
                    label = "{} / {}".format(n.tally_count - count, label)
                else:
                    color = 'pink2'
            style = 'filled'
            if len(n.props): style += ', bold'
            nodeid = f"n{id(n)}"
            G.add_node(nodeid, label=label, style=style, fillcolor=color)
            G.add_edge(p, nodeid)
            add_nodes(nodeid, n.children)
    for l, matcher in dag.children.items():
        nodeid = f"name_{id(l)}"
        G.add_node(nodeid, label=f'"{l}"')
        if matcher.wildcard:
            add_nodes(nodeid, [matcher.wildcard])
        for v, nodes in matcher.positive_values.items():
            nodeid2 = f"value_{id(v)}"
            G.add_node(nodeid2, label=f'"{v}"', style='filled', fillcolor='lightyellow', shape='box')
            G.add_edge(nodeid, nodeid2)
            add_nodes(nodeid2, nodes)
        # TODO handle negative values here
    
    return Image(nx.nx_agraph.to_agraph(G).draw(prog='dot',format='png'))

Here we build a single dag containing both clauses and formulae, but the design is easier to understand if we regard them as two separate graphs. Consider the graph of clauses. The invariant is that a node represenging a clause $c$ needs to be reachable from exactly those root nodes representing literals $l \in c$. It must be reachable via some path from each of those root nodes, and not reachable from any others.

Subject to this, and without being too precise, the goal is to minimize the number of edges and also to minimize 
fan-out from any node. Intuitively, we want to minimize the duplication of work for overlapping formulae. Also, since we have no reason to believe that any particular fact will become true in the future, we also want to minimize the expected amount of work we have to do for any incremental addition of bindings to our assignment, deferring as much work as possible.

To build the dag, we build the whole thing in two phases, first adding all clauses from smallest to largest, then adding all formulae likewise. For this reason, we require the complete set of formulae up-front. We use the greedy set-cover approximation algorithm for each phase. NB: This is where we rely on the fact that clauses and formulae are both ordered from smallest to largest.

For reasons that will become clear below, we force every literal to build a disjunction node, but single-element clauses and formulae can simply reuse the node below them as their own.

In [ ]:
from ccs.dag import build_dag

With this in place, we can build an entire rule tree. In the future, if we're really just going to immediately flatten the tree to a list of those nodes containing primitive rules, there's really no need to build the tree at all. But see below under **Finding additional sharing?** for a reason we may want to keep the trees around... So for now, we'll just flatten and sort it here.

In [ ]:
draw_tree(root)

In [ ]:
draw_dag(build_dag(filter(lambda n: len(n.props) + len(n.constraints), root)))

Just to play around, we'll want a utility for going straight from expression strings to rule tree nodes that can be built into a dag.

In [ ]:
def exprs(*strs):
    return [RuleTreeNode(formula=to_dnf(parse_expr(str))) for str in strs]

In [ ]:
list(x.formula for x in exprs('(a, b) (a, c)', 'a b c'))

Let's take a look at the result:

In [ ]:
draw_dag(build_dag(exprs('a b c', 'a b', 'b c (e, f, g)', 'b d (e, f)', 'b')))

#### Examples

Here are a few more little examples from my notes, just to show a few features of the way these dags are built.

In [ ]:
draw_dag(build_dag(exprs('a b c', 'a b c')))

In [ ]:
draw_dag(build_dag(exprs('a', 'b', 'c', 'a b', 'b c', 'a b c')))

In [ ]:
draw_dag(build_dag(exprs('a', 'b', 'c', 'a b', 'b c', 'a b c', 'a c', 'a c d')))

In [ ]:
draw_dag(build_dag(exprs('a b c d', 'a b d', 'a c d', 'b d', 'a b')))

In [ ]:
draw_dag(build_dag(exprs('b c (e, f, g)')))

#### Finding additional sharing? 

This algorithm will never create an intermediate node, so it'll only discover sharing among expressions that are actually present in the input. Compare the following:

In [ ]:
draw_dag(build_dag(exprs('a b c', 'b c d')))

In [ ]:
draw_dag(build_dag(exprs('a b c', 'b c d', 'b c')))

But our input files themselves often contain strong hints about that, such as:

    b c {
      a {...}
      d {...}
    }
    
So we could get some mileage by exploiting these structures when we build the list of formulae.

We'll parse the above into a tree of formulae, all normalized:

    b c
     a b c
     b c d
        
For each branch, we can check whether the parent is a subset of more than one of its children. If so, we add the parent formula to our set. Otherwise, we drop it.

So, for

    b {
      c {
        a {...}
        d {...}
      }
    }
    
we find this tree:

    b
      b c
       a b c
       b c d

We eliminate `b`, but retain `b c`.

In [ ]:
n = parse_only("""
b {
  c {
    a : x = 1
    d : x = 2
  }
}
""")
root = RuleTreeNode()
n.add_to(root)
draw_tree(root)

In [ ]:
draw_dag(build_dag(exprs('a b c', 'b c d')))

In [ ]:
draw_dag(build_dag(exprs('b c', 'a b c', 'b c d')))

For

    (a, b) {
      a {...}
      c {...}
    }
    
we find

    ab d
     a d
     ab c d

and since `ab d` is a subset of only one of its children, we eliminate it, saving one node.

Note that there are cases where this still builds unwanted intermediate nodes. For instance:

    a b c {...}
    a b {
      c d {...}
      c e {...}
    }

The `a b` node will be included in this case, optimistically hoping that it'll be used for `a b c d` and `a b c e`, but in fact both of those will take advantage of `a b c`, leaving `a b` with no other children. Truly orphaned nodes and nodes with only one child and no settings/assertions like that could just be ignored, they could be cleaned out at the end, or maybe the above subset heuristic could be improved to detect them sooner.

In [ ]:
n = parse_only("""
a b c {x = 1}
a b {
  c d {x = 2}
  c e {x = 3}
}
""")
root = RuleTreeNode()
n.add_to(root)
draw_tree(root)

In [ ]:
draw_dag(build_dag(exprs('a b c', 'a b c d', 'a b c e')))

In [ ]:
draw_dag(build_dag(exprs('a b', 'a b c', 'a b c e', 'a b c e')))

_**Question:** Does this section up to here still make sense when we're building DNF rather than CNF?_

In any case, this is a possible extension for later.

Here's an example that shows the value of this:

In [ ]:
draw_dag(build_dag(exprs('a b c d e (f, g, h, i, j)')))

#### Matching

Meanwhile, let's talk about matching against these dags. This part is simple. We just start at the correct root for the newly added literal fact, and activate whichever nodes we reach, decrementing the tally count of each. We activate a node's children only when its tally count reaches *exactly* zero. This prevents duplicate activations in the case of disjunctions. (It would also allow an easy way to poison nodes, if ever needed.) We also need to be careful not to allow duplicate literals, but forcing each single-element clause to build a one-input disjunction node accomplishes that as well.

Finally, we keep the tally counts in a persistent map so that we can fork them as discussed above.

#####  Specificity

As far as specificity is concerned, there are three types of nodes:

 1. disjunction nodes representing literals
 2. conjunction nodes
 3. true disjunction nodes representing formulae

In the case of 1, activation specificity is constant, and the node only needs to be activated once.

In the case of 2, with a bit of care when building literal nodes, we can calculate the exact specificity as we build the dag and store it in the node. So the node only needs to be activated at most once, and specificity is known.

In the case of 3, the activation specificity is always the specificity propagated from the activating node, and activation occurs not only the first time, but whenever to newly propagated specificity is greater than any previous activation specificity.

This suggests that we need two node types, with distinct activation policies, one for cases 1 and 2, and one for case 3. The tally state for the first node type is just the current activation count, and the tally state for the second node type is the last activation specificity, if any.

In [ ]:
from ccs.search_state import Context

In reality, there will be some extra wrinkles pertaining to the more complex real-world structure of literals, property settings, specificities, rule-activated fact augmentations, and so on. But it's all fairly straightforward after this.

In [ ]:
dag = build_dag(exprs('a b c', 'a b', 'b c (e,f,g)', 'b d (e,f)', 'b'))
root = Context(dag)
c = root.augment('b')

Let's take a look at the state of a graph in context as well:

In [ ]:
draw_dag(dag, c.tallies)

In [ ]:
draw_dag(dag, c.augment('a').tallies)

In [ ]:
draw_dag(dag, c.augment('e').tallies)

In [ ]:
from functools import reduce
def augmentAll(c, ls):
    return reduce(lambda c, l: c.augment(l), ls, c)
draw_dag(dag, augmentAll(root, 'bdcg').tallies)

As a reminder, the rules for these examples were

    exprs('a b c', 'a b', 'b c (e,f,g)', 'b d (e,f)', 'b')

so those activations are exactly as expected.

This example shows the necessity of tracking dirty literals:

In [ ]:
draw_dag(dag, augmentAll(root, 'aa').tallies)

And this example shows why we build set-valued literal nodes as tally-1 conjunctions:

In [ ]:
n = parse_only("""
(foo.a, foo.b, foo.c) a b : x = 123
""")
rule_tree = RuleTreeNode()
n.add_to(rule_tree)
dag = build_dag(filter(lambda n: len(n.props) + len(n.constraints), rule_tree))
root = Context(dag)
draw_dag(dag, root.augment('foo', 'a').tallies)

Rather than forcing an indirection of every literal through a disjunction node, we could also just track dirty literals explicitly. But the current approach is probably better, since we need a node on which to hang settings and constraints in any case. And it's more consistent and simpler, even if it makes the pictures a little uglier.

### Property values

In [ ]:
import pyrsistent

n = parse_only("""
a, f b e, c {
  c d {
    x = y
  }
  e f {
    foobar = abc
  }
}
a, c, b e f : baz = quux
""")
rule_tree = RuleTreeNode()
n.add_to(rule_tree)
dag = build_dag(filter(lambda n: len(n.props) + len(n.constraints), rule_tree))
root = Context(dag)

def check(keys):
    ctx = augmentAll(root, keys)
    print(pyrsistent.thaw(ctx.props))
    return draw_dag(dag, ctx.tallies)

In [ ]:
check('b')

In [ ]:
check('aef')

In [ ]:
check('cd')

In [ ]:
check('feb')

### Quick exercise of CCS1 override functionality

In [ ]:
n = parse_only("""
env.dev day.today {
   foo = right
   bar = wrong
}
env.dev {
  foo = wrong
  @override bar = right
}
""")
rule_tree = RuleTreeNode()
n.add_to(rule_tree)
dag = build_dag(filter(lambda n: len(n.props) + len(n.constraints), rule_tree))
root = Context(dag)

ctx = root.augment('day', 'today').augment('env', 'dev')
assert list(ctx.props['foo'].values)[0].value == 'right'
assert list(ctx.props['bar'].values)[0].value == 'right'

### Real files!

In [ ]:
tmp1 = parser.Parser().parse(io.StringIO("""
(a.x, a.y, a.z) b c d : foo = bar
a c : foo = baz
"""), '-')
tmp2 = RuleTreeNode()
tmp1.add_to(tmp2)
tmp3 = build_dag(filter(lambda n: len(n.props)+len(n.constraints), tmp2))

In [ ]:
import os.path

class ImportResolver:
    def __init__(self, basedir):
        self.basedir = basedir
        
    def resolve(self, location):
        if not os.path.isabs(location):
            location = os.path.join(self.basedir, location)
        if os.path.isfile(location):
            return open(location)
        raise ValueError(f"No such file: '{location}'")

In [ ]:
resolver = ImportResolver('tmp')

%time tmp1 = parser.Parser().parse_ccs_stream(resolver.resolve('main.ccs'), 'main.ccs', resolver, [])
tmp2 = RuleTreeNode()
%time tmp1.add_to(tmp2)
%time tmp3 = build_dag(filter(lambda n: len(n.props)+len(n.constraints), tmp2))

In [ ]:
#draw_tree(tmp2)

In [ ]:
#draw_dag(tmp3)

In [ ]:
print(tmp2.stats())
tmp3.stats().dump()

In [ ]:
def ctx_from_file(dag, fname, *, poison=False):
    with open(fname) as f:
        path = f.read()
    root = Context(dag, poisoned=pyrsistent.s() if poison else None)

    import re
    steps = re.findall(r"([A-Za-z0-9_-]+(\.[A-Za-z0-9_-]+)?)", path)
    ctx = root
    for step in steps:
        pair = step[0].split('.')
        ctx = ctx.augment(pair[0], pair[1] if len(pair) > 1 else None)
        
    return ctx
        
ctx = ctx_from_file(tmp3, 'example_ctx1.txt')

In [ ]:
pyrsistent.thaw(ctx.props)

In [ ]:
#draw_dag(tmp3, ctx.tallies, True)

## Canonical dumping

In [ ]:
# here's the idea behind this one... we want to dump all the property 
# settings with the normalized formula for each. we want to be able to 
# exclude settings which can never match in the current context. this
# isn't done yet, but it means we'll want to operate on the dag plus
# tally state, rather than on the rule tree, which would otherwise probably
# be simpler.

# we top sort the dag depth-first starting at each literal node. as we 
# visit each literal, we propagate its literal into the expression
# being built for the destination node. then all that remains is to scan
# the nodes in top sort order, propagating each node's clause/formula
# onward to its children and dumping any properties present in the node
# itself. we must do this in two phases because the literal sets in the
# initial nodes of the top sort aren't fully populated until the initial
# scan of the literals is finished (and same with the other dag nodes, in
# fact).

# finally we sort first by property name, then by formula, and print the
# results.

def top_sort(dag):
    visited = set()
    result = []
    
    def visit(n):
        if n in visited:
            return
        for m in n.children:
            visit(m)
        visited.add(n)
        result.append(n)

    node_forms = {}
    
    def visit_literal_node(node, name, value=None):
        # at this point, we know all the nodes are going to be
        # AndNodes, but we also know that in actual fact they correspond
        # to disjunctions of literals, so we do something a bit special
        # to build the clause:
        assert isinstance(node, AndNode)
        
        # TODO this won't really work right for disjunctions of a wildcard
        # plus actual values (as in '(a, a.x, a.y) : foo = bar'), but i'm
        # pretty sure that's already broken other places as well. anyway,
        # add a test and fix it everywhere!
        if node in node_forms:
            values = node_forms[node].first().values
        else:
            values = set()
        to_add = {value} if value else set()
        node_forms[node] = Clause([Key(name, values | to_add)])
        visit(node)
        
    for l, matcher in dag.children.items():
        if matcher.wildcard:
            visit_literal_node(matcher.wildcard, l)
        for v, nodes in matcher.positive_values.items():
            for node in nodes:
                visit_literal_node(node, l, v)
        # TODO handle negative values here        
        
    return reversed(result), node_forms


# TODO this is terrible and wants a cleanup!
def combine(f1, f2):
    if isinstance(f1, Clause) and isinstance(f2, Clause):
        return f1.union(f2)
    if isinstance(f1, Formula) and isinstance(f2, Formula):
        return Formula(f1.clauses.union(f2.clauses))
    if isinstance(f1, Clause) and isinstance(f2, Formula):
        return Formula(f2.clauses | {f1})
    assert False, f"what are you trying to do? {type(f1)} {type(f2)}"

    
import sys    
def dump_dag(ctx, *, out=sys.stdout):
    dag = ctx.dag
    poisoned = ctx.poisoned or pyrsistent.s()
    nodes, node_forms = top_sort(dag)
    
    results = []
    for node in nodes:
        # TODO is this the correct place to bail out here? or only when we add the props to result? think hard about this!
        if node in poisoned:
            continue
        form = node_forms[node]
        for prop in node.props:
            # TODO this is terrible, find a better way to do it. in general,
            # wouldn't it be easier to just store the normalized formula
            # in each node?? how much memory could that possibly really waste?
            prop_form = Formula([form]) if isinstance(form, Clause) else form
            results.append((prop_form, prop)) # TODO include origin!
        # TODO also handle constraints!
        for child in node.children:
            if child in node_forms:
                child_form = node_forms[child]
            else:
                child_form = Clause([]) if isinstance(child, AndNode) else Formula([])
            node_forms[child] = combine(form, child_form)

    def sort_key(result):
        return (result[1][0], result[0])
        
    for result in sorted(results, key=sort_key):
        name = result[1][0]
        prop = result[1][1]
        ovd = "@override " if prop.override_level > 0 else ""
        print(f"{result[0]} : {ovd}{name} = '{prop.value}' // {prop.origin}", file=out)

In [ ]:
with open('tmp/asdf6.ccs', 'w') as out:
    dump_dag(ctx_from_file(tmp3, 'example_ctx3.txt', poison=True), out=out)

These can be diffed as in:

https://stackoverflow.com/questions/44019/an-easy-way-to-diff-log-files-ignoring-the-time-stamps


Here's a side-by-side diff, ignoring the property origins, the works even with Mac OS's crappy sed. (There's probably an even better way to handle the newline insertion on MacOS sed, but this is good enough to prove the point...)

    diff -y -I '^  //' <(sed  -E 's:^(.*)//:\1#  //:' foo.dump | tr \# \\n) <(sed  -E 's:^(.*)//:\1#  //:' bar.dump | tr \# \\n)

And meld can do it easily with one of its default 'file filters'!

## What's next

 - override/underride!
 - conflict detection/resolution by property number
 - negative matches!
 - unset!
 - clean up, port and update tests

Notes:
 - conflicts should be reported when queried, not when loaded, probably even in case of two settings in the exact same dag node?
 - add test for multiple values for same key augmented at the same time.
 - should we allow something like `foo > foo.bar` or is that a disallowed duplicate value?
 - we *should* allow multiple assertions of the same fact, like `foo.bar > foo.bar`. This allows us to add known implications to the files themselves, which can help simplify the output of dump/diff by pruning things that are known to be irrelevant, but not necessarily yet present in the input context.